In [1]:
import polars
import hvplot.polars

import holoviews as hv
import utils
import indicatory.names as names

from polars import col

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
amd = utils.load_asset_data("AMD")
amd.head()

Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Time Zone
date,f64,f64,f64,f64,i64,f64,f64,str
1980-03-17,0.0,3.302083,3.125,3.145833,219600,0.0,0.0,"""EST"""
1980-03-18,0.0,3.125,2.9375,3.03125,727200,0.0,0.0,"""EST"""
1980-03-19,0.0,3.083333,3.020833,3.041667,295200,0.0,0.0,"""EST"""
1980-03-20,0.0,3.0625,3.010417,3.010417,159600,0.0,0.0,"""EST"""
1980-03-21,0.0,3.020833,2.90625,2.916667,130800,0.0,0.0,"""EST"""


In [3]:
import indicatory.oscillators as osc

In [13]:
short = 5
long = 20
ppo = osc.percentage_price_oscillator(amd, short_window_size=short, long_window_size=long)
simple_rsi = osc.simple_relative_strength_index(amd)
rsi = osc.relative_strength_index(amd)
so = osc.stochastic_oscillator(amd)
dpo = osc.detrend_price_oscillator(amd)
roc_close = osc.rate_of_change(amd, names.CLOSE)
roc_rsi = osc.rate_of_change(rsi, names.rsi(10))

In [9]:
elements = 100
periods = elements - long
ohlc = utils.ohlc_plot(amd.tail(periods))
ppo_plot = utils.scatter_plot(ppo.tail(periods), value_fields=[names.ppo(column_name="Close", short_window_size=short, long_window_size=long)])
hline = hv.HLine(0).options(color='black', line_width=1)

hv.Layout(ohlc + ppo_plot * hline).cols(1) 

:Layout
   .Overlay.I  :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Overlay.II :Overlay
      .Scatter.I :Scatter   [Date]   (PPO 5/20 Close)
      .HLine.I   :HLine   [x,y]

In [10]:
overbought_threshold = hv.HLine(70).options(line_width=1, color="red")
oversold_threshold = hv.HLine(30).opts(line_width=1, color="red")
simple_rsi_plot = utils.scatter_plot(simple_rsi.tail(periods), value_fields=[names.rsi(10)])
rsi_plot = utils.scatter_plot(rsi.tail(periods), value_fields=[names.rsi(10)], color="darkblue")

hv.Layout(ohlc + simple_rsi_plot * overbought_threshold * oversold_threshold + rsi_plot * overbought_threshold * oversold_threshold).cols(1)

:Layout
   .Overlay.I   :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Overlay.II  :Overlay
      .Scatter.I :Scatter   [Date]   (RSI 10)
      .HLine.I   :HLine   [x,y]
      .HLine.II  :HLine   [x,y]
   .Overlay.III :Overlay
      .Scatter.I :Scatter   [Date]   (RSI 10)
      .HLine.I   :HLine   [x,y]
      .HLine.II  :HLine   [x,y]

In [11]:
so_fast_plot = utils.step_plot(so.tail(periods), value_fields=[names.fast_k(5), names.fast_d(5)], color=["green", "orange"])
so_slow_plot = utils.step_plot(so.tail(periods), value_fields=[names.slow_k(5), names.slow_d(5)], color=["blue", "purple"])

hv.Layout(ohlc + so_fast_plot * overbought_threshold * oversold_threshold + so_slow_plot * overbought_threshold * oversold_threshold).cols(1)

:Layout
   .Overlay.I   :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Overlay.II  :Overlay
      .NdOverlay.I :NdOverlay   [Variable]
         :Curve   [Date]   (value)
      .HLine.I     :HLine   [x,y]
      .HLine.II    :HLine   [x,y]
   .Overlay.III :Overlay
      .NdOverlay.I :NdOverlay   [Variable]
         :Curve   [Date]   (value)
      .HLine.I     :HLine   [x,y]
      .HLine.II    :HLine   [x,y]

In [12]:
dpo_plot = utils.bar_plot(dpo.tail(periods), value_field=names.dpo(column_name="Close", window_size=10))

hv.Layout(ohlc + dpo_plot).cols(1)

:Layout
   .Overlay.I :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Bars.I    :Bars   [Date]   (DPO 10 Close)

In [12]:
roc_close_plot = utils.bar_plot(roc_close.tail(periods), value_field=names.roc(column=names.CLOSE))
roc_rsi_plot = utils.bar_plot(roc_rsi.tail(periods), value_field=names.roc(column=names.rsi(10)))

hv.Layout(ohlc + roc_close_plot + rsi_plot + roc_rsi_plot).cols(1)

:Layout
   .Overlay.I :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Bars.I    :Bars   [Date]   (roc Close)
   .Scatter.I :Scatter   [Date]   (RSI 10)
   .Bars.II   :Bars   [Date]   (roc RSI 10)